#importing required packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from keras.preprocessing import image
import sqlite3
from datetime import date

getting current time to store it in patient's data

In [2]:
from datetime import datetime
now = datetime.now()
curr_time = now.strftime("%d/%m/%Y %H:%M:%S")
print(curr_time)

04/08/2021 14:53:32


Creating a csv file to store patients data 

In [ ]:
import csv
with open("database1.csv","w+") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Name","Time","FilePath","Result"])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Performing image augmentation for train data

In [4]:
train_datagen=image.ImageDataGenerator(
    rescale = 1/255,horizontal_flip=True,zoom_range=0.2,shear_range=0.2
)
train_data =train_datagen.flow_from_directory(directory="/content/drive/MyDrive/CovidDetection2/CovidDataset/Train",target_size=(256,256),batch_size=16,class_mode="binary")

Found 224 images belonging to 2 classes.


In [5]:
train_data.class_indices

{'Covid': 0, 'Normal': 1}

Performing image augmentation for test data

In [6]:
test_datagen=image.ImageDataGenerator(
    rescale = 1/255
)
test_data =test_datagen.flow_from_directory(directory="/content/drive/MyDrive/CovidDetection2/CovidDataset/Val",target_size=(256,256),batch_size=16,class_mode="binary")

Found 60 images belonging to 2 classes.


#Cnn model

In [7]:
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",input_shape=(256,256,3)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",))
model.add(MaxPool2D())
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation="relu"))
model.add(Dropout(rate=0.50))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,optimizer="adam",metrics=['acc'])



In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 252, 252, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 126, 126, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 64)        0

In [9]:
model.fit_generator(train_data,steps_per_epoch=8,epochs=10,validation_steps=2,validation_data= test_data)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 104s 11s/step - loss: 1.8297 - acc: 0.5460 - val_loss: 0.6922 - val_acc: 0.5312
Epoch 2/10
8/8 [==============================] - 59s 7s/step - loss: 0.6758 - acc: 0.5961 - val_loss: 0.6729 - val_acc: 0.7500
Epoch 3/10
8/8 [==============================] - 51s 6s/step - loss: 0.6370 - acc: 0.6421 - val_loss: 0.5520 - val_acc: 0.8750
Epoch 4/10
8/8 [==============================] - 51s 6s/step - loss: 0.4568 - acc: 0.7562 - val_loss: 0.3664 - val_acc: 0.8750
Epoch 5/10
8/8 [==============================] - 50s 6s/step - loss: 0.3854 - acc: 0.8498 - val_loss: 0.2321 - val_acc: 0.9688
Epoch 6/10
8/8 [==============================] - 50s 6s/step - loss: 0.2856 - acc: 0.8877 - val_loss: 0.2299 - val_acc: 0.9688
Epoch 7/10
8/8 [==============================] - 50s 6s/step - loss: 0.2327 - acc: 0.9289 - val_loss: 0.1771 - val_acc: 0.9688
Epoch 8/10
8/8 [==============================] - 50s 6s/step - loss: 0.2231 - acc: 0.8964 - val_loss:

Selecting image for diagnosis displaying result and appending it in csv file

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from keras.preprocessing import image
from google.colab import files
import csv
name=str(input("Enter Patients name : "))
uploaded = files.upload()

for fn in uploaded.keys():
        path=fn
        img=image.load_img(path,target_size=(256,256))
        img=image.img_to_array(img)/255
        image=np.array(img)/255
        img=np.array([img])
        img.shape
print(model.predict_classes(img))
result=model.predict_classes(img)
def stored_result(result):
      if result == 0:
          return "Covid-Positive"
      else:
          return "Covid-Negative"
print(stored_result(result))
with open("database1.csv","a") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([name,curr_time,path,stored_result(result)])

Enter Patients name : Max


Saving NORMAL2-IM-0380-0001.jpeg to NORMAL2-IM-0380-0001.jpeg


/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[1]]
Covid-Negative
